In [1]:
import sqlite3
import pandas as pd

In [3]:

conn = sqlite3.connect("birdinfo.db")
try: 
    conn.execute("PRAGMA foreign_keys = ON") #ugh this took me forever to figure out
    conn.execute("""
    DROP TABLE IF EXISTS sites
    """)
    conn.execute("""
    DROP TABLE IF EXISTS observers
    """)
    conn.execute("""
    DROP TABLE IF EXISTS nets
    """)
    conn.execute("""
    DROP TABLE IF EXISTS birds
    """)
    conn.execute("""
    DROP TABLE IF EXISTS effort
    """)
    
    #observers can exist without ever doing any work/catching any birds
    #only one observer per ID
    
    conn.execute("""
    CREATE TABLE observers (id INT, name TEXT, PRIMARY KEY ('id'));
    """)
    
    conn.execute("""
    INSERT INTO observers VALUES (1,'Heather'), (2, 'Sam'), 
    (3, 'Zane'),  (4, 'Sasha')""")
    
    #You need an observer to put in effort
    # each observer can only have one entry per day
    conn.execute("""
    CREATE TABLE effort (date TEXT, tech INT, hours INT,
    FOREIGN KEY ('tech') REFERENCES observers ('id')
    ON DELETE CASCADE
    ON UPDATE CASCADE,
    PRIMARY KEY ('date', 'tech'));
    """)
    
    conn.execute("""
    INSERT INTO effort VALUES ('May1', 2, 4), ('May1', 1, 3), 
    ('May2', 2, 4), ('May2', 3, 4), ('May3', 4, 4), ('May3', 1, 4), ('May4', 1, 1), ('May4', 2, 5)
    """)
    
    # Sites are unique
    conn.execute("""
    CREATE TABLE sites (siteID TEXT, size INT,
    PRIMARY KEY ('siteID'))
    """)
    
    conn.execute("""
    INSERT INTO sites VALUES ('Alberts', 20), ('Bear', 25), ('Bould', 50) 
    """)
    
    
    #nets can exist at known sites; each net name is unique
    conn.execute("""
    CREATE TABLE nets (name TEXT, W_utm INT, N_utm INT, site TEXT,
    PRIMARY KEY ('name'),
    FOREIGN KEY ('site') 
    REFERENCES sites ('siteID')
    ON DELETE CASCADE
    ON UPDATE CASCADE)
    """)
    
    #these UTMS make no sense but that's okay 
    conn.execute("""
    INSERT INTO nets VALUES ("BB-A1", 282249,3882304, "Bould"), ("BB-A2", 283249,3881304, "Bould"),("BB-A3", 282449,3812304, "Bould"),
    ("BB-A4", 289249,3882374, "Bould"), ("BB-A5", 282249,3887304, "Bould"), ("ALB-A1", 282249,3882303, "Alberts"), ("ALB-A2", 282241,3882303, "Alberts"),
    ("ALB-B1", 282049,3882304, "Alberts"), ("ALB-B2", 282449,3882304, "Alberts"), ("ALB-B3", 282229,3882104, "Alberts"),
    ("LB-B1", 282009,3882304, "Bear")""")
    
    #can't catch a bird without an observer or a date but you can catch birds twice
    #observers can't catch birds on days they weren't working 
    conn.execute("""
    CREATE TABLE birds (bandnumber INT, species TEXT, sex TEXT, observer INT, mass INT, net TEXT, date TEXT,
    FOREIGN KEY ('observer', 'date')
    REFERENCES effort ('tech', 'date')
    ON DELETE CASCADE
    ON UPDATE CASCADE)
    """)
    
    conn.execute("""
    INSERT INTO birds VALUES (511025, 'BTBW', 'M', 1, 5, "BB-A1", 'May1'), 
    (511026, 'BTBW', 'M', 1, 3, "BB-A2", 'May1'), (511027, 'HOWA', 'M', 1, 2,"BB-A1", 'May1'), 
    (511028, 'BTBW', 'F', 4, 2, "BB-A3", 'May3'), (511029, 'CAWA', 'M', 2, 2,"ALB-A1", 'May4'), 
    (511030, 'CAWA', 'M', 2, 4, "BB-A3", 'May2'), (511031, 'CAWA', 'F', 3, 4,"ALB-B1", 'May2'),
    (511031, 'CAWA', 'F', 1, 4,"ALB-A1", 'May1'), (511031, 'CAWA', 'F', 1, 4,"ALB-A1", 'May3'),
    (511034, 'BTBW', 'M', 2, 4,"ALB-A2", 'May4'), (511210, 'HOWA', 'F', 3, 4,"ALB-A2", 'May2')
    """)
    conn.commit();
    
except sqlite3.Error as e:
        print("ERROR: "+str(e))

finally:
    print("done")
    conn.close()

done
